# Imports

In [ ]:
import numpy as np
import os,sys
import six.moves.urllib as urllib
import sys
import tarfile
import zipfile
import imageio
imageio.plugins.ffmpeg.download()

from datetime import datetime
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
RESEARCH_PATH= '../../tf-models/researvh'
MODELS_PATH='../../tf-models/research/object_detection'
sys.path.append(RESEARCH_PATH)
sys.path.append(MODELS_PATH)

## Env setup

In [ ]:
%matplotlib inline

## Object detection imports

In [ ]:
from utils import label_map_util
from utils import visualization_utils as vis_util

## Model preparation

In [ ]:
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

PATH_TO_CKPT = MODEL_NAME +'/frozen_inference_graph.pb'

PATH_TO_LABELS = os.path.join('data','mscoco_label_map.pbtxt')

NUM_CLASSES = 90

## Download Model

In [ ]:
opener = urllib.request.URLopener()
opener.retreive(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
    file_name = os.path.basename(file.name)
    if 'frozen_inference_graph.pb' in file_name:
        tar_file.extract(file, os.getcwd())


## Load a frozen Tensorflow model into memory.

In [ ]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

## Loading label map

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [ ]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

## Detection

In [ ]:
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [os.path.join(PATH_TO_TEST_IMAGES_DIR,'beach{}.jpg'.format(i)) for i in range(1, 3)]
IMAGE_SIZE = (12,8)

In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor=detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes=detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores=detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes=detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections=detection_graph.get_tensor_by_name('num_detections:0')
        
        input_video = 'traffic'
        video_reader=imageio.get_reader('%s.mp4'%input_video)
        video_writer=imageio.get_writer('%s_annotated.mp4'%input_video,fps =10)
        #loop through each frame
        t0 = datetime.now()
        n_frames = 0
        for frame in video_reader:
            image_np=frame
            n_frames+=1
            
            #expanddim since code expects images to have shape[1,None]
            image_np_expanded=np.expand_dims(image_np,axis=0)
            #Actual detection
            (boxes,scores,classes,num)=sess.run([detection_boxes,detection_scores,detection_classes,num_detections,feed_d])
            #visualisation
            vis_util.visualize_boxes_and_labels_on_image_array(image_np,no.squeeze(boxes),np.squeeze(classes).astype(np.int))
            #video_writer
            video_writer.append_data(image_np)
        
        fps = n_frames/datetime.now()-t0)total_seconds()
        print("Frames processed: %s,Speeed:%s fps"%(n_frames,fps))
        #cleanup
        video_writer.close()